#MATCHSUM_Kor version

-Extractive Summarization as Text Matching([paper](https://https://arxiv.org/pdf/2004.08795.pdf))

##- Dependencies

- kobert-transformers           0.5.1
- transformers                  4.25.1
- konlpy                        0.6.0(mecab)
- pytorch-metric-learning       1.6.3
- torch                         1.13.0

##- Data
AI-HUB 문서요약 텍스트(신문기사, 기고문, 잡지기사, 법원 판결문 원문, 요약 3문장)[Link]((https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97))

##- 설명

- 본 코랩파일은 간단하게 구동할 수 있도록 정리되어있습니다. 예시로 봐주세요.

### - 시작하기 전에
- kor_rouge_metric.py 파일이 필요합니다. github에서 다운받은 후 구글 드라이브에 업로드하여 저장경로를 입력해주세요.
- matchsum 모델은 사전에 요약모델을 사용한 pruning 과정이 필요합니다. 즉, 요약할 문서에서 중요한 문장 5개의 인덱스가 저장되어 있는 파일이 필요합니다.(없는 경우 제가 생성한 예시파일을 [이곳](https://drive.google.com/file/d/1-WOL5LUwQW4srDMMBJS6HP0FZBt3EYg9/view?usp=share_link)에서 다운로드(약 600MB)하여 사용해주세요.)

In [ ]:
#경로 설정
rouge_path = 'kor_rouge_metric.py 파일 경로' #ex)'/content/drive/MyDrive/matchsum'
data_path = 'data 파열 경로(.json)' #ex)'/content/drive/MyDrive/matchsum/prun_data.json'
output_dir = '모델 savepoint 경로' #ex)'/content/drive/MyDrive/matchsum/savepoint'
model_path = 'model weight 경로' #ex)'/content/drive/MyDrive/matchsum/model/test_model_5.pt'

In [3]:
#kor_rouge_metric를 사용하기 위해 path 추가
import sys
sys.path.append(rouge_path) # kor_rouge_metric.py 저장 경로를 입력해주세요.

In [ ]:
#mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
from konlpy.tag import Mecab

mecab = Mecab()
mecab.morphs('동해물과 백두산이 마르고 닳도록')

In [ ]:
!pip install kobert_transformers

In [ ]:
#Metric base
!pip install pytorch_metric_learning

In [7]:
import pandas as pd
import json
import re
from kor_rouge_metric import Rouge #py 파일이 별도로 필요
from transformers import BertModel
from kobert_transformers import get_tokenizer
from pytorch_metric_learning.losses import BaseMetricLossFunction
from torch.optim import Adam, AdamW
from transformers import EarlyStoppingCallback, TrainingArguments,Trainer
import os
from os.path import join
from time import time
import torch
from torch import nn
from torch.nn import init
from datetime import timedelta
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

In [ ]:
tokenizer = get_tokenizer() #kobert의 토크나이저
bert_model = BertModel.from_pretrained('monologg/kobert')

#Candidates Pruning
KoBERTSUM 사용
(https://github.com/uoneway/KoBertSum)

# Candidate combination

- custom dataset을 사용하여 candidate하는 경우 이 부분을 실행해주세요.

In [ ]:
#초기 설정
data = #데이터프레임 형태의 데이터
tokenizer = get_tokenizer() #kobert의 토크나이저
save_path = #pruning된 데이터 저장할 경로

In [69]:
Final_rouge = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,
        )

In [75]:
#get_candidate
from itertools import combinations

def get_candidates(tokenizer, idx,df):
    MAX_LEN=180
    MAX_LEN_text = 512

    # load data
    data = {}
    #original text data
    data['text'] = df.loc[idx]['src_txt']
    #gold summary
    data['summary'] = df.loc[idx]['abstractive']

    # get candidate summaries
    #각각 문서에서 5개의 중요한 문장을 truncate합니다.(using koBertSum)
    # 2,3개의 문장을 선택하여 combinations을 사용하여 C(5,2)+C(5,3)=20 총 20개의 후보군 요약문을 생성합니다.
    # 본 모델은 기본적으로 koBertSum 모델을 사전에 활용하여야 get candidate를 할 수 있습니다.
    # 데이터프레임(df)에 ['sum_sents_idxes']열에 5개의 문장인덱스가 존재해야합니다.
    #pruned text(5) index
    sent_id = df.loc[idx]['sum_sents_idxes']
    indices = list(combinations(sent_id, 2))
    indices += list(combinations(sent_id, 3))
    sep = '[SEP]'
    sep_id = tokenizer.encode(sep, add_special_tokens=False)

    # get ROUGE score for each candidate summary and sort them in descending order
    score = []
    for i in indices:
        i = list(i)
        i.sort()

        # write dec
        dec = []
        for j in i:
            sent = data['text'][j]
            dec.append(sent)
        #dec는 candidate summary당 문장 모음
        rouge = Final_rouge.get_scores(df.loc[idx]['sum_sents_tokenized'], dec)

        rouge1 = float(rouge['rouge-1']['f'])
        rouge2 = float(rouge['rouge-2']['f'])
        rougel = float(rouge['rouge-l']['f'])
        rouge = (rouge1 + rouge2 + rougel) / 3
        score.append((i, rouge))
        
    score.sort(key=lambda x : x[1], reverse=True)
    
    # write candidate indices and score
    data['ext_idx'] = sent_id
    data['indices'] = []
    data['score'] = []
    for i, R in score:
        data['indices'].append(list(map(int, i)))
        data['score'].append(R)

    # tokenize and get candidate_id
    candidate_summary = []
    for i in data['indices']:
        cur_summary = []
        for j in i:
            cur_summary.append(data['text'][j])
        cur_summary = ' '.join(cur_summary)
        candidate_summary.append(cur_summary)
        
    data['candidate_summary'] = candidate_summary

    return data

In [80]:
# 데이터에서 candidate를 실행
import time
import datetime
prun_data = []

start = time.time() 
for idx in range(len(data)):
    prun_data.append(get_candidates(tokenizer,idx,data))
    if idx%1000 ==0:
      sec = time.time()-start # 종료 - 시작 (걸린 시간)
      times = str(datetime.timedelta(seconds=sec))
      print(f"{idx}processing time : {times} sec")
times = str(datetime.timedelta(seconds=sec))
print(f"end time : {times} sec")

0processing time : 0:00:00.459500 sec
1000processing time : 0:04:17.681429 sec
2000processing time : 0:08:02.693164 sec
3000processing time : 0:11:37.180689 sec
4000processing time : 0:15:15.015369 sec
5000processing time : 0:18:49.685857 sec
6000processing time : 0:22:26.528919 sec
7000processing time : 0:26:10.695462 sec
8000processing time : 0:29:57.360230 sec
9000processing time : 0:33:41.452856 sec
10000processing time : 0:37:31.184223 sec
11000processing time : 0:41:26.886138 sec
12000processing time : 0:45:29.492182 sec
13000processing time : 0:49:27.924716 sec
14000processing time : 0:53:22.644642 sec
15000processing time : 0:57:16.140237 sec
16000processing time : 1:01:05.287196 sec
17000processing time : 1:05:12.737635 sec
18000processing time : 1:09:17.862427 sec
19000processing time : 1:13:19.831756 sec
20000processing time : 1:17:18.421368 sec
21000processing time : 1:21:24.934761 sec
end time : 1:21:24.934761 sec


In [ ]:
#데이터를 저장
prun_data =pd.DataFrame(prun_data)
pruning_data = pd.merge(data, prun_data,left_index=True, right_index=True)
pruning_data.to_json(join(save_path, 'prun_data.json', orient = 'table')

,src_txt,tgt_txt,sum_sents_tokenized,sum_sents_idxes,abstractive
0,[대한민국 5 G 홍보 대사 를 자처 한 문재 인 대통령 은 넓 고 체증 없 는 통...,대한민국 5 G 홍보 대사 를 자처 한 문재 인 대통령 은 넓 고 체증 없 는 통신...,문 대통령 은 8 일 서울 올림픽 공원 에서 열린 5 G 플러스 전략 발표 에 참석...,"[1, 0, 9, 6, 4]",[8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 ...
1,"[8 일 서울 올림픽 공원 K 아트홀, 지난 3 일 한국 이 세계 최초 로 5 세대...",지난 3 일 한국 이 세계 최초 로 5 세대 5 G 이동 통신 서비스 를 상용 화 ...,8 일 서울 올림픽 공원 K 아트홀\n문재인 대통령 홍남기 부총리 겸 기획 재정부 ...,"[0, 8, 10, 6, 1]",[지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리...
2,[] 박원순 서울 시장 사진 이 8 일 고층 재 개발 재건축 관련 요구 에 작심 한...,] 박원순 서울 시장 사진 이 8 일 고층 재 개발 재건축 관련 요구 에 작심 한 ...,박 시장 은 이날 서울 시청 에서 열린 골목길 재생 시민 정책 대화 에 참석 해 과...,"[1, 0, 4, 3, 5]",[박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하...
3,"[SK 주 와 미국 알파벳 구글 지주회사 의 간결 한 지배 구조 를 배워라, 기업 ...",기업 지배 구조 개선 등 을 통해 높 은 수익 률 을 올리 는 것 을 목표 로 하 ...,SK 주 와 미국 알파벳 구글 지주회사 의 간결 한 지배 구조 를 배워라\nKB 운...,"[0, 2, 1, 7, 9]",[주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적...
4,[MBC 당신 이 믿 었 던 페이크 가 JTBC 손석희 대표 이사 의 동승자 논란 ...,MBC 당신 이 믿 었 던 페이크 가 JTBC 손석희 대표 이사 의 동승자 논란 진...,MBC 당신 이 믿 었 던 페이크 가 JTBC 손석희 대표 이사 의 동승자 논란 진...,"[0, 8, 10, 9, 5]",[MBC ' 당신이 믿었던 페이크' 가 JTBC 손석희 대표의 '동승자 논란' 진실...
...,...,...,...,...,...
21301,"[울릉도 명이 가 유명세 를 탈 전망 이 다, 29 일 한국 을 공식 방문 한 도널...",울릉도 명이 가 유명세 를 탈 전망 이 다<q>29 일 한국 을 공식 방문 한 도널...,29 일 한국 을 공식 방문 한 도널드 트럼프 대통령 의 청와대 친교 만찬 에 경북...,"[1, 2, 0, 9, 8]",[포항시는 오는 7월 당초 109개 노선 200대 운행에서 119개 노선 270대로...
21302,[경기도 수원 의 카페 투어 를 소재 로 한 토크 쇼 가 일본 의 수도 도쿄 에서 ...,경기도 와 경기 관광공사 는 지난 29 일 일본 도쿄 에서 경기도 골목 의 카페 와...,경기도 와 경기 관광공사 는 지난 29 일 일본 도쿄 에서 경기도 골목 의 카페 와...,"[1, 0, 8, 6, 9]",[경북울릉군 특산물인 명이장아찌가 2017년 한·미 정상회담 독도새우에 이어 29일...
21303,[산업은행 은 다음 달 23 24 일 이틀 간 서울 강남구 코엑스 에서 넥스트 라이...,산업은행 은 다음 달 23 24 일 이틀 간 서울 강남구 코엑스 에서 넥스트 라이즈...,산업은행 은 다음 달 23 24 일 이틀 간 서울 강남구 코엑스 에서 넥스트 라이즈...,"[0, 7, 8, 10, 1]",[지난 해 일본어 핸드 가이드 북인 '수원 카페북(마루마루 수원)이 투어리즘 엑스포...
21304,[경기도 는 집배원 택배 기사 등 이동 노동자 들 을 위한 폭염 대책 의 하나 로 ...,경기도 는 집배원 택배 기사 등 이동 노동자 들 을 위한 폭염 대책 의 하나 로 경...,경기도 는 집배원 택배 기사 등 이동 노동자 들 을 위한 폭염 대책 의 하나 로 경...,"[0, 8, 6, 9, 4]",[산은은 국내외 벤처생태계 저명인사들을 기조연설자로 초청해 국내 벤처생태계 활성화에...


#MATCHSUM
- 예시 데이터를 사용할 경우 이 부분부터 실행하시면 됩니다.



### Class 및 함수 정의
- CustomDataset (class : 데이터셋 정의)
- custom_collate_fn (함수 : Trainer 내부에 collate_fn)
- MatchSum (Class : 모델 정의)
- train_model (함수 : train 파이프라인 정의)
- test_model (함수 : test 파이프라인 정의)
- ValidMetric (Class : evaluate metric 정의)
- custom_compute_metrics (함수 : evaluate 파이프라인)
- MatchRougeMetric (Class : Test metric 정의)
- custom_compute_metrics_test (함수 : test 파이프라인)
- MyTrainer (Class : huggingface Trainer custom)

In [8]:
#json 불러오기
with open(data_path) as f: 
    prun_data = json.load(f)
    prun_data = prun_data['data']
    prun_data =pd.DataFrame(prun_data)

###utils

In [9]:
class CustomDataset(Dataset):
    
    def __init__(self, input_data1:list,input_data2:list,input_data3:list) -> None:
        self.X = input_data1
        self.Y = input_data2
        self.Z = input_data3
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index], self.Z[index]

In [10]:
def custom_collate_fn(batch):
    """
    - batch: list of dictionary 
    
    한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
    이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
    
    (input1,input2,input3, target) 딕셔너리 형태를 반환.
    """
    max_len=180
    max_len_text=512 #원래 512


    text_list, cand_list, summ_list = [], [], []
    token_cand = []
    for input_text, input_cand, input_summ in batch:
        text_list.append(input_text)
        cand_list.append(input_cand)
        summ_list.append(input_summ)
    
    tensorized_text = tokenizer(
    text_list,
    add_special_tokens=True,
    padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
    truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
    max_length=max_len_text,
    return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
    
    for candidate in cand_list:
        tensorized_cand = tokenizer(
        candidate,
        add_special_tokens=True,
        padding='max_length',  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가 ("longest")
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=max_len,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
        )

        token_cand.append(tensorized_cand)

    tensorized_summ = tokenizer(
    summ_list,
    add_special_tokens=True,
    padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
    truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
    max_length=max_len,
    return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )

    #Trainer의 evaluate()를 위한 더미 변수
    labels= torch.rand(len(tensorized_summ), 1)
    
    result ={'text_id':tensorized_text,'candidate_id': token_cand, 'summary_id':tensorized_summ,'labels':labels}
    return result

In [11]:
#candidate개수 조정(20개->8개)
prun_data['candidate_summary'] = prun_data['candidate_summary'].apply(lambda x: x[:8])
prun_data['indices'] = prun_data['indices'].apply(lambda x: x[:8])

#data split
dataset_size = len(prun_data)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

#evaluate 부분에서 reset된 index가 필요
#eval_train_dataset = prun_data[:train_size].reset_index(drop=True)
eval_validation_dataset = prun_data[train_size:train_size+validation_size].reset_index(drop=True)
eval_test_dataset = prun_data[train_size+validation_size:].reset_index(drop=True)

# text, summary 문자열 합쳐주기
prun_data['text'] = prun_data['text'].apply(lambda x: ' '.join(x))
prun_data['summary'] = prun_data['summary'].apply(lambda x: ' '.join(x))

#train에 사용할 데이터셋
train_dataset = prun_data[:train_size].reset_index(drop=True)
validation_dataset = prun_data[train_size:train_size+validation_size].reset_index(drop=True)
test_dataset = prun_data[train_size+validation_size:].reset_index(drop=True)

print('train :',train_size, len(train_dataset))
print('validation :',validation_size,len(validation_dataset))
print('test :',test_size,len(test_dataset))

#Dataset 설정
train_set = CustomDataset(train_dataset['text'], train_dataset['candidate_summary'], train_dataset['summary'])
validation_set = CustomDataset(validation_dataset['text'], validation_dataset['candidate_summary'], validation_dataset['summary'])
test_set = CustomDataset(test_dataset['text'], test_dataset['candidate_summary'], test_dataset['summary'])

train : 17044 17044
validation : 2130 2130
test : 2132 2132


###Model, train, test, Trainer

In [12]:
class MatchSum(nn.Module):
    
    def __init__(self, candidate_num, encoder, hidden_size=768):
        super(MatchSum, self).__init__()
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        
        self.encoder = BertModel.from_pretrained(encoder)

    def forward(self, text_id, candidate_id, summary_id, labels):
        
        batch_size = text_id['input_ids'].size(0)

        # get document embedding
        outputs = self.encoder(input_ids=text_id['input_ids'], attention_mask=text_id['attention_mask'],
                           token_type_ids=text_id['token_type_ids'])
        
        last_hidden_states = outputs[0]
        doc_emb = last_hidden_states[:,0,:]

        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]
        
        # get summary embedding
        outputs = self.encoder(input_ids=summary_id['input_ids'], attention_mask=summary_id['attention_mask'],
                           token_type_ids=summary_id['token_type_ids'])
        last_hidden_states = outputs[0]
        summary_emb = last_hidden_states[:, 0, :]

        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1)

        # get candidate embedding
        cand_input_ids = []
        cand_attention_mask = []
        cand_token_type_ids = []

        for candidate in candidate_id:
            cand_input_ids.append(candidate['input_ids'])
            cand_attention_mask.append(candidate['attention_mask'])
            cand_token_type_ids.append(candidate['token_type_ids'])
        # stack embeddings
        cand_input_ids = torch.stack(cand_input_ids).view(-1,len(cand_input_ids[0][0]))
        cand_attention_mask = torch.stack(cand_attention_mask).view(-1,len(cand_attention_mask[0][0]))
        cand_token_type_ids = torch.stack(cand_token_type_ids).view(-1,len(cand_token_type_ids[0][0]))


        outputs = self.encoder(input_ids=cand_input_ids, attention_mask=cand_attention_mask,
                           token_type_ids=cand_token_type_ids)
        last_hidden_states = outputs[0]
        candidate_emb = last_hidden_states[:, 0, :].view(batch_size, self.candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]        
        
        assert candidate_emb.size() == (batch_size, self.candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)

        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        assert score.size() == (batch_size, self.candidate_num)

        #score는 문장단위 score , summary_score는 요약 단위 score
        return {'score': score, 'summary_score': summary_score}

In [34]:
def train_model(train, valid,args):
        
    # load summarization datasets
    print('Information of dataset is:')
    train_dataset = train
    valid_dataset = valid

    # configure training
    devices = [0]
    params = {}
    params['encoder']       = 'monologg/kobert'
    params['candidate_num'] = 8
    params['batch_size']    = 4 #32
    params['accum_count']   = 2
    params['max_lr']        = 2e-5
    params['margin']        = 0.01
    params['warmup_steps']  = 3000
    params['n_epochs']      = 1
    params['valid_steps']   = 1000
    train_params=params

    print('Devices is:')
    print(devices)

    # configure model
    model = MatchSum(params['candidate_num'] , train_params['encoder'])

    optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8)

    assert 16 % len(devices) == 0
    #train(model, train_set)
    trainer = MyTrainer(model= model,args=args,train_dataset=train_dataset, eval_dataset=valid_dataset,
                        compute_metrics= custom_compute_metrics,callbacks=[EarlyStoppingCallback(early_stopping_patience = 1)],
                      optimizers=(optimizer,None),data_collator=custom_collate_fn,loss_name='MarginRankingLoss',
                      margin = 0.01 )
    print('Start training with the following hyper-parameters:')
    print(train_params)
    trainer.train()
    
    torch.save(model, join(model_path,'model.pt'))

In [13]:
def test_model(dataset, models_path,args_test,args):

    print('Information of dataset is:')
    print(dataset)
    test_dataset = dataset
    
    # need 1 gpu for testing
    device = [0]
    
    batch_size = 1
        
    # load model
    model = torch.load(models_path)

    optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8)

    # configure testing
    tester = MyTrainer(model= model,args=args_test,train_dataset=train_dataset, eval_dataset=test_dataset,
                    compute_metrics= custom_compute_metrics_test,callbacks=[EarlyStoppingCallback(early_stopping_patience = 1)],
                  optimizers=(optimizer,None),data_collator=custom_collate_fn,loss_name='MarginRankingLoss',
                  margin = 0.01 )
    tester.evaluate()

In [14]:
class MyTrainer(Trainer):
    # loss_name 이라는 인자를 추가로 받아 self에 각인 시켜줍니다.
    def __init__(self, loss_name, margin, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_name= loss_name # 각인!
        self.margin = margin

    def compute_loss(self, model, inputs, return_outputs=False):

        # config에 저장된 loss_name에 따라 다른 loss 계산 
        if self.loss_name == 'MarginRankingLoss':
            # lossname이 MarginRankingLoss 이면, custom_loss에 torch.nn.MarginRankingLoss()를 선언(?) 해줍니다.
            custom_loss = torch.nn.MarginRankingLoss(self.margin)
  
        outputs = model(**inputs)

        score = outputs['score']
        summary_score = outputs['summary_score']

        # here is to avoid that some special samples will not go into the following for loop
        ones = torch.ones(score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss = loss_func(score, score, ones)

        # candidate loss
        n = score.size(1)
        for i in range(1, n):
            pos_score = score[:, :-i]
            neg_score = score[:, i:]
            pos_score = pos_score.contiguous().view(-1)
            neg_score = neg_score.contiguous().view(-1)
            ones = torch.ones(pos_score.size()).cuda(score.device)
            loss_func = torch.nn.MarginRankingLoss(self.margin * i)
            TotalLoss += loss_func(pos_score, neg_score, ones)

        # gold summary loss
        pos_score = summary_score.unsqueeze(-1).expand_as(score)
        neg_score = score
        pos_score = pos_score.contiguous().view(-1)
        neg_score = neg_score.contiguous().view(-1)
        ones = torch.ones(pos_score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss += loss_func(pos_score, neg_score, ones)
        
        return (TotalLoss, outputs) if return_outputs else TotalLoss

### metrics

In [27]:
#for evaluate
class ValidMetric(BaseMetricLossFunction):  
    def __init__(self, save_path, data, score=None):
        super(ValidMetric, self).__init__()
        #self._init_param_map(score=score)
 
        self.save_path = save_path
        self.data = data

        self.top1_correct = 0
        self.top6_correct = 0
        #self.top10_correct = 0
         
        self.rouge = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,)
        self.ROUGE = 0.0
        self.Error = 0

        self.cur_idx = 0
    
    # an approximate method of calculating ROUGE
    def fast_rouge(self, dec, ref):

        if dec == '' or ref == '':
            return 0.0
        scores = self.rouge.get_scores(dec, ref)

        return (scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3

    def evaluate(self, score):

        batch_size = score.size(0)

        #score에서 각 차원의 최대값 인덱스를 뽑아내는 것.
        self.top1_correct += int(torch.sum(torch.max(score, dim=1).indices == 0))
        self.top6_correct += int(torch.sum(torch.max(score, dim=1).indices <= 5))
        self.top10_correct += int(torch.sum(torch.max(score, dim=1).indices <= 9))

        # Fast ROUGE
        for i in range(batch_size):
            max_idx = int(torch.max(score[i], dim=0).indices)
            if max_idx >= len(self.data.loc[self.cur_idx]['indices']):
                self.Error += 1 # Check if the candidate summary generated by padding is selected
                self.cur_idx += 1
                continue
            ext_idx = self.data.loc[self.cur_idx]['indices'][max_idx]
            ext_idx.sort()
            dec = []
            ref = ''.join(self.data.loc[self.cur_idx]['summary'])
            for j in ext_idx:
                dec.append(self.data.loc[self.cur_idx]['text'][j])
            dec = ''.join(dec)

            self.ROUGE += self.fast_rouge(dec, ref)
            self.cur_idx += 1

    def get_metric(self, reset=True):

        top1_accuracy = self.top1_correct / self.cur_idx
        top6_accuracy = self.top6_correct / self.cur_idx
        top10_accuracy = self.top10_correct / self.cur_idx

        ROUGE = self.ROUGE / self.cur_idx


        eval_result = {'top1_accuracy': top1_accuracy, 'top6_accuracy': top6_accuracy,'top10_accuracy': top10_accuracy, 
                       'Error': self.Error, 'ROUGE': ROUGE} 
       
        with open(join(self.save_path, 'train_info.txt'), 'a') as f:
            print('top1_accuracy = {}, top6_accuracy = {},top10_accuracy={},Error = {}, ROUGE = {}'.format(
                top1_accuracy, top6_accuracy,top10_accuracy,self.Error, ROUGE),file=f)

        if reset:
            self.top1_correct = 0
            self.top6_correct = 0
            self.top10_correct = 0
            self.ROUGE = 0.0
            self.Error = 0
            self.cur_idx = 0
        return eval_result

In [28]:
#for test
class MatchRougeMetric(BaseMetricLossFunction):
    def __init__(self, data, n_total,dec_path, save_name, score=None, save =False):
        super(MatchRougeMetric, self).__init__()
        self.data        = data
        self.n_total     = n_total
        self.cur_idx = 0
        self.ext = []
        self.start = time()
        self.rouge = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,)

        self.path = dec_path
        self.save_name = save_name
        self.save = save

    def evaluate(self, score):

        batch_size = score.size(0)

        for i in range(batch_size):
            ext = int(torch.max(score[i], dim=0).indices) # batch_size = 1
            self.ext.append(ext)
            print('{}/{} ({:.2f}%) decoded in {} seconds\r'.format(
                  i, self.n_total, self.cur_idx/self.n_total*100, timedelta(seconds=int(time()-self.start))
                ), end='')
    
    def save_output(self, dec_list, path, name):
        with open(join(self.path, '{}.txt'.format(self.name)), 'w') as f:
            print(self.dec_list, file=f)

    def get_metric(self, reset=True):
        R_1_sum = 0
        R_2_sum = 0
        R_L_sum = 0
        print('\nStart calculate each text !!!')
        for i, ext in enumerate(self.ext):
            sent_ids = self.data.loc[i]['indices'][ext]
            dec=[]
            for j in sent_ids:
                dec.append(self.data.loc[i]['text'][j])#self.cur_idx

            #저장하는 메소드 만들기
            if self.save == True:
                self.save_output(dec, self.path, self.save_name)           
            
            dec = ''.join(dec)
            ref = self.data.loc[i]['summary']
        
            self.cur_idx += 1

            R_1, R_2, R_L = self.eval_rouge(dec, ref)
        
            R_1_sum += R_1
            R_2_sum += R_2
            R_L_sum += R_L
        
        R_1_mean = R_1_sum/self.cur_idx   #self.n_total
        R_2_mean = R_2_sum/self.cur_idx 
        R_L_mean = R_L_sum/self.cur_idx 

        print('Start evaluating ROUGE score !!!')
        
        eval_result = {'ROUGE-1': R_1_mean, 'ROUGE-2': R_2_mean, 'ROUGE-L':R_L_mean} 
        
        if reset == True:
            self.cur_idx = 0
            self.ext = []
            self.data = []
            self.start = time()

        return eval_result
        
    def eval_rouge(self, dec, ref):
        if dec == '' or ref == '':
            return 0.0
        scores = self.rouge.get_scores(dec, ref)
        R_1 = scores['rouge-1']['f']
        R_2 = scores['rouge-2']['f']
        R_L = scores['rouge-l']['f']

        return R_1, R_2, R_L

In [29]:
#for evaluate - compute_metrics
def custom_compute_metrics(pred):
    preds = pred.predictions[0]
    preds = torch.Tensor(preds)
    val_metric = ValidMetric(save_path='/content/drive/MyDrive/matchsum/savepoint', data=eval_validation_dataset)
    val_metric.evaluate(preds)
    eval_result = val_metric.get_metric()
    return eval_result

In [30]:
#for test - compute_metrics
def custom_compute_metrics_test(pred):
    preds = pred.predictions[0]
    preds = torch.Tensor(preds)
    test_metric = MatchRougeMetric(data=eval_test_dataset, n_total = len(eval_test_dataset),dec_path=dec_path, save_name='save1')
    test_metric.evaluate(preds)
    eval_result = test_metric.get_metric()
    return eval_result

### set Trainer arguments

In [39]:
#arguments setting

args_train = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy= 'epoch', #
    load_best_model_at_end=True, # train 종료시 best model 로드할지 여부

    # Dataset
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,

    # Evaluation 
    evaluation_strategy = "epoch",# 각 epoch 마지막에 평가

    # Randomness
    seed=42,

    #for ealry stopping callback
    metric_for_best_model = 'ROUGE',

    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
)

args_test = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy= 'epoch',
    load_best_model_at_end=True, # train 종료시 best model 로드할지 여부

    # Dataset
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    
    metric_for_best_model ='ROUGE-1',
    
    # Evaluation 
    evaluation_strategy = "epoch",# 각 epoch 마지막에 평가

    # Randomness
    seed=42,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


###실행

In [ ]:
#모델 train
train_model(train_set, validation_set, args_train)

In [ ]:
#모델 test
test_model(test_set, model_path,args_test)